El código a continuación debe ser ejecutado en el propio ArcGIS con su entorno de desarrollo, si no, no se ejecutará correctamente (única forma de sacar imágenes de su visión activa).

In [1]:
from PIL import Image, ImageDraw
import geopandas as gpd
import arcpy
import os
import time
import json
import keyboard
import shapely.geometry
import glob
import shutil
import random

# Ruta del proyecto de ArcGIS Pro
project_path = r"C:\Users\hugoh\OneDrive\Documentos\ArcGIS\Projects\TFG_palmeras\TFG_palmeras.aprx"

# Ruta del archivo Shapefile con las palmeras
shapefile_path = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\Mapa_IDECanarias_Palmeras/99_K_Mapapalmerascanarias.shp"

# Carpeta de salida para las imágenes
output_folder = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\fixed_normalization"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Comprobamos las coordenadas del shapefile y de la vista activa de cámara

In [3]:
# Cargar las coordenadas de las palmeras
data = gpd.read_file(shapefile_path)

# Abrir el proyecto de ArcGIS Pro
project = arcpy.mp.ArcGISProject("current")
map_view = project.activeView

if map_view == None:
    raise ValueError("No se encontró ninguna vista activa en ArcGIS Pro. Abre el mapa manualmente y vuelve a ejecutar el script.")

Chequeamos el sistemas de las coordenadas y convertimos al mismo de la vista activa para unificar valores

In [3]:
# --- Asegurar que el mapa está en UTM Zona 28N ---
sr_utm = arcpy.SpatialReference(32628)
camera = map_view.camera
camera.spatialReference = sr_utm
map_view.camera = camera
print("Sistema de coordenadas fijado a UTM Zona 28N (EPSG:32628)")
print("Sistema de coordenadas después del cambio", map_view.camera.getExtent().spatialReference)
# --- Convertir las palmeras a UTM si fuera necesario ---
data_gc = data[data["ISLA"] == "GRAN CANARIA"]  # Filtrar solo Gran Canaria
print(data_gc.head())
print("Revisando sistema de coordenadas de las palmeras...")
if data_gc.crs.to_epsg() != 32628:
    print("Convirtiendo palmeras a UTM Zona 28N (EPSG:32628)...")
    data_gc = data_gc.to_crs(epsg=32628)
print("Sistema de coordenadas de las palmeras confirmado: UTM Zona 28N (EPSG:32628)")

Sistema de coordenadas fijado a UTM Zona 28N (EPSG:32628)
Sistema de coordenadas después del cambio <geoprocessing spatial reference object object at 0x0000025CB65F3570>
                 Id_palm Tip_amb                       Tip_amb_de Hibrid  \
132957  Cx454646y3100878       ?  Ambientes de difícil asignación   None   
132958  Cx454653y3100874       ?  Ambientes de difícil asignación   None   
132959  Cx454659y3100870       ?  Ambientes de difícil asignación   None   
132960  Cx454677y3100862       ?  Ambientes de difícil asignación   None   
132961  Cx454672y3100865       ?  Ambientes de difícil asignación   None   

                Hibrid_de          ISLA      MUNICIPIO  \
132957  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   
132958  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   
132959  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   
132960  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   
132961  Palmeras canarias  GRAN CANARIA  SANTA BRÍGIDA   

                        

Funciones que convierten las coordenadas en objetos PointGeometry para comparar si se encuentran dentro de una extensión con poligon geometry.

In [21]:
# Función para comprobar si una coordenada está dentro del extent exportado
def is_in_exported_image(x_real, y_real, extent,  img_width, img_height):
    """"
    x_real, y_real: coordenadas en el mapa de la palmera
    extent: extent del mapa exportado
    :return: (bool, x_pixel, y_pixel) donde x_pixel y y_pixel son las coordenadas de la palmera u objeto en píxeles
    """
    x_pixel = ((x_real - extent.XMin) / extent.width) * img_width
    y_pixel = ((extent.YMax - y_real) / extent.height) * img_height

    inside = (0 <= x_pixel <= img_width) and (0 <= y_pixel <= img_height)
    return inside, x_pixel, y_pixel



In [ ]:
# Verificar si existen palmeras con valor en el campo "Hibrid"
palmeras_con_hibrid = data_gc[~data_gc["Hibrid"].isna()]

if not palmeras_con_hibrid.empty:
    print(f"Se encontraron {len(palmeras_con_hibrid)} palmeras con 'Hibrid' distinto de null.")
    print(palmeras_con_hibrid[["Hibrid", "Hibrid_de"]].head())
else:
    print("No se encontraron palmeras con 'Hibrid' distinto de null.")


In [28]:
print("Longitud dataset gc", len(data_gc))

Longitud dataset gc 176430


Creación del dataset de imágenes para las detecciones, junto a sus anotaciones y etiquetas.

In [ ]:

folders = {
    "train": os.path.join(output_folder, "Train"),
    "val": os.path.join(output_folder, "Val"),
    "test": os.path.join(output_folder, "Test")
}

for folder in folders.values():
    os.makedirs(os.path.join(folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(folder, "labels_ambiente"), exist_ok=True)
    os.makedirs(os.path.join(folder, "labels_palmera"), exist_ok=True)
    os.makedirs(os.path.join(folder, "labels_binaria"), exist_ok=True)
    os.makedirs(os.path.join(folder, "json_annotations"), exist_ok=True)
    os.makedirs(os.path.join(folder, "with_bbox"), exist_ok=True)
    os.makedirs(os.path.join(folder, "worldfiles"), exist_ok=True)
    os.makedirs(os.path.join(folder, "label_number_class"), exist_ok=True)

# Inicialización de ArcGIS
project = arcpy.mp.ArcGISProject("CURRENT")
# Vista activa del mapa
map_view = project.activeView
# Camara del mapa
camera = map_view.camera
# UTM zona 28N (Canarias)
sr_utm = arcpy.SpatialReference(32628)
camera.spatialReference = sr_utm
map_view.camera = camera

# Se carga y convierte el shapefile a las mismas coordenadas que el mapa
data = data.to_crs(epsg=32628) if data.crs.to_epsg() != 32628 else data
islas = {"train": "GRAN CANARIA", "val": "TENERIFE", "test": "LA GOMERA"}
data_split = {k: data[data["ISLA"] == v].copy() for k, v in islas.items()}

# Se obtienen clases únicas de ambiente e híbrido
data_gc = data_split["train"]
data_gc["Hibrid"] = data_gc["Hibrid"].fillna("pc")
ambiente_classes = {k: i for i, k in enumerate(sorted(data_gc["Tip_amb"].dropna().unique()))}
palmera_classes = {k: i for i, k in enumerate(sorted(data_gc["Hibrid"].unique()))}

# Guardar las clases en fichero labels.txt para cada carpeta
for subset, folder in folders.items():
    with open(os.path.join(folder, "labels_ambiente", "labels.txt"), "w", encoding="utf-8") as f:
        for name in ambiente_classes:
            f.write(f"{name}\n")
    with open(os.path.join(folder, "labels_palmera", "labels.txt"), "w", encoding="utf-8") as f:
        for name in palmera_classes:
            f.write(f"{name}\n")
    with open(os.path.join(folder, "labels_binaria", "labels.txt"), "w", encoding="utf-8") as f:
        f.write("palmera\n")
    with open(os.path.join(folder, "label_number_class", "ambiente_classes.txt"), "w", encoding="utf-8") as f:
        for name, idx in ambiente_classes.items():
            f.write(f"{idx}: {name}\n")
    with open(os.path.join(folder, "label_number_class", "palmera_classes.txt"), "w", encoding="utf-8") as f:
        for name, idx in palmera_classes.items():
            f.write(f"{idx}: {name}\n")

# Parámetros comunes
resolution_m_per_px = 0.25
box_size = 45
map_scale = 1000

# Creación de la función para exportar anotaciones
def export_annotation(image_name, palms, extent, width, height, folder):
    json_path = os.path.join(folder, "json_annotations", image_name.replace(".png", ".json"))
    with open(json_path, "w", encoding="utf-8") as jf:
        json.dump({"image": image_name, "extent": extent.JSON, "palms": palms}, jf, indent=4)

    label_path_amb = os.path.join(folder, "labels_ambiente", image_name.replace(".png", ".txt"))
    label_path_hyb = os.path.join(folder, "labels_palmera", image_name.replace(".png", ".txt"))
    label_path_bin = os.path.join(folder, "labels_binaria", image_name.replace(".png", ".txt"))

    # Mover worldfile
    pgw_path = image_name.replace(".png", ".pgw")
    pgw_src = os.path.join(folder, "images", pgw_path)
    pgw_dst = os.path.join(folder, "worldfiles", pgw_path)
    if os.path.exists(pgw_src):
        os.replace(pgw_src, pgw_dst)

    with open(label_path_amb, "w", encoding="utf-8") as f_amb, \
         open(label_path_hyb, "w", encoding="utf-8") as f_hyb, \
         open(label_path_bin, "w", encoding="utf-8") as f_bin:
        with open(pgw_dst) as wf:
            # Parámetros del worldfile, explicados en la memoria del TFG
            A, D, B, E, C, F = [float(wf.readline()) for _ in range(6)]

        # Escribir anotaciones en formato YOLO
        for palm in palms:
            # Coordenadas geográficas
            x_geo, y_geo = palm["real_coordinates"]["x"], palm["real_coordinates"]["y"]

            # Dimensiones del contenedor
            w, h = palm["bbox_pixel"]["width"], palm["bbox_pixel"]["height"]

            # Se calculan coordenadas en píxeles y se normalizan para el formato YOLO
            x_px = int((x_geo - C) / A)
            y_px = int((y_geo - F) / E)
            x_norm = float(x_px) / width
            y_norm = float(y_px) / height
            w_norm = float(w) / width
            h_norm = float(h) / height

            amb = ambiente_classes.get(palm["attributes"].get("Tip_amb"), -1)
            hyb = palmera_classes.get(palm["attributes"].get("Hibrid", "pc"), -1)

            if amb != -1:
                f_amb.write(f"{amb} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            if hyb != -1:
                f_hyb.write(f"{hyb} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            f_bin.write(f"0 {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    # Dibujar contenedores en la imagen para la visualización de las mismas
    img_path = os.path.join(folder, "images", image_name)
    if os.path.exists(img_path) and os.path.exists(pgw_dst):
        img = Image.open(img_path).copy()
        draw = ImageDraw.Draw(img)
        with open(pgw_dst) as wf:
            A, D, B, E, C, F = [float(wf.readline()) for _ in range(6)]
        for palm in palms:
            x_geo, y_geo = palm["real_coordinates"]["x"], palm["real_coordinates"]["y"]
            w = palm["bbox_pixel"]["width"]
            h = palm["bbox_pixel"]["height"]
            x_px = int((x_geo - C) / A)
            y_px = int((y_geo - F) / E)
            draw.rectangle([(x_px - w//2, y_px - h//2), (x_px + w//2, y_px + h//2)], outline="red", width=2)
        img.save(os.path.join(folder, "with_bbox", image_name))

# Función para procesamiento por conjunto, generando imágenes y anotaciones
def process_island(data_island, folder, max_palms):
    # Set para evitar duplicados (palmeras ya procesadas)
    processed = set()
    index = 0
    counter = 0
    while (index < len(data_island) and counter < max_palms):
        if keyboard.is_pressed('esc'):
            print("Ejecución interrumpida manualmente con ESC.")
            break
        
        # Se recorre en bloques de 40, saltando de 400 en 400 (explicado el por qué en la memoria del TFG)
        for offset in range(40):
            if index + offset >= len(data_island):
                break
            # Se obtiene la fila actual dentro del dataframe para obtener el índice de la palmera para el dataframe (shapefile) completo original
            row = data_island.iloc[index]
            # Índice de la palmera
            idx = row.name
            if idx in processed:
                continue
            
            # Se posiciona la cámara en la ubicación de la palmera
            camera.X, camera.Y = row.geometry.x, row.geometry.y
            camera.scale = map_scale
            map_view.camera = camera
            time.sleep(1)

            # Se obtiene el extent del mapa actual, es decir, el área visible de la cámara
            extent = map_view.camera.getExtent()
            # Ancho y alto del extent en metros
            w_m = extent.XMax - extent.XMin
            h_m = extent.YMax - extent.YMin
            # Ancho y alto del extent en píxeles
            width_px = int(w_m / resolution_m_per_px)
            height_px = int(h_m / resolution_m_per_px)

            # Nombre de la imagen a exportar con el identificador de la palmera en la que se centra la cámara
            image_name = f"palmera_{idx}.png"
            image_path = os.path.join(folder, "images", image_name)
            # Se exporta la vista actual a imagen PNG con worldfile
            map_view.exportToPNG(image_path, width_px, height_px, world_file=True)

            # Se crea un polígono rectangular, en este caso, que representa el área visible de la cámara
            box = shapely.geometry.box(extent.XMin, extent.YMin, extent.XMax, extent.YMax)
            # Lista de palmeras dentro del área visible
            palms = []
            # Se recorren todas las palmeras que están dentro del área visible (extent)
            for j, palm_row in data_island[data_island.geometry.within(box)].iterrows():
                if j in processed:
                    continue
                # Coordenadas de la palmera reales
                x, y = palm_row.geometry.x, palm_row.geometry.y

                # Conversion a píxeles dentro del extent (imagen exportada)
                x_px = ((x - extent.XMin) / w_m) * width_px
                y_px = ((extent.YMax - y) / h_m) * height_px

                # Creamos la anotacion para cada palmera
                palms.append({
                    "tree_id": j,
                    "real_coordinates": {"x": x, "y": y},
                    "attributes": palm_row.drop("geometry").to_dict(),
                    "bbox_pixel": {""
                        "x_center": int(x_px), 
                        "y_center": int(y_px), 
                        "width": box_size, 
                        "height": box_size
                    },
                    "image_size_px": (width_px, height_px)
                })
                processed.add(j)
        # Si se detectaron palmeras dentro de esta imagen, se exportan sus anotaciones a cada carpeta correspondiente, creandose así los ficheros de anotaciones YOLO con estos datos
        if palms:
            export_annotation(image_name, palms, extent, width_px, height_px, folder)
            counter += len(palms)
        # Se avanza el índice en 400 para saltar a la siguiente palmera, procesando así el siguiente bloque de 40 palmeras
        index += 400
    return counter

# Ejecutar procesamiento en orden Train/Val/Test
total_gc = process_island(data_split["train"], folders["train"], float("inf"))

# Si las palmeras de gc representan el 70% del total, el total será ese numero más un 30% del mismo
total_palms_dataset = total_gc + (total_gc * 0.3)
val_count = int(total_palms_dataset * 0.2)
test_count = int(total_palms_dataset * 0.1)
total_tnf = process_island(data_split["val"], folders["val"], val_count)
total_gomera = process_island(data_split["test"], folders["test"], test_count)

print("Dataset completo generado con la estructura Train/Val/Test.")
print(f"Numero de palmeras de GC: {total_gc}")
print(f"Numero de palmeras de TNF: {total_tnf}")
print(f"Numero de palmeras de LA GOMERA: {total_gomera}")
print(f"Numero de palmeras totales: {total_palms_dataset} y suma que debe ser lo mismo {total_gc + total_tnf + total_gomera}")

# Se muestra el recuento de imágenes y anotaciones YOLO por conjunto
def count_annotations(folder_name):
    image_dir = os.path.join(folder_name, "images")
    label_dir = os.path.join(folder_name, "labels_binaria")
    num_images = len([f for f in os.listdir(image_dir) if f.endswith(".png")])
    num_labels = len([f for f in os.listdir(label_dir) if f.endswith(".txt")])
    return num_images, num_labels

print("\nResumen del dataset generado:")
for split in ["train", "val", "test"]:
    img_count, label_count = count_annotations(folders[split])
    print(f"{split.capitalize()}: {img_count} imágenes | {label_count} anotaciones")


Captura de imágenes para el conjunto de datos de clasificación de ambiente

In [ ]:
output_folder = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\environment_classification_dataset"
os.makedirs(output_folder, exist_ok=True)

# Directorio donde se guardarán las capturas procesadas
capture_folder = os.path.join(output_folder, "CapturesRaw")
os.makedirs(capture_folder, exist_ok=True)

folders = {
    "train": os.path.join(output_folder, "Train"),
    "val": os.path.join(output_folder, "Val"),
    "test": os.path.join(output_folder, "Test")
}

islands = ["GRAN CANARIA", "TENERIFE", "LA GOMERA"]

data_islands = data[data["ISLA"].isin(islands)].copy()
data_env = data_islands[data_islands["Tip_amb"].notna()].copy()

# Rellenar los nulos con "pc"
data_islands["Hibrid"] = data_islands["Hibrid"].fillna("pc")
data_type = data_islands.copy()

# Crear diccionarios de clases
env_classes = {k: i for i, k in enumerate(sorted(data_env["Tip_amb"].unique())) if k != "?"}
type_classes = {k: i for i, k in enumerate(sorted(data_type["Hibrid"].unique())) if k != "?"}


print("env_classes: ", env_classes)
print("type_classes: ", type_classes)


# Parámetros comunes
resolution_m_per_px = 0.25
scale = 1000
img_size = 256
max_per_class = 600
# Todas las imágenes se guardarán en la carpeta de entrenamiento y luego se dividirán en Train/Val/Test
train_folder = folders["train"]

# Reproyección de las coordenadas de las palmeras a UTM Zona 28N
data_env = data_env.to_crs(epsg=32628)

# Conexión con ArcGIS Pro
project = arcpy.mp.ArcGISProject("CURRENT")
map_view = project.activeView
camera = map_view.camera
camera.spatialReference = arcpy.SpatialReference(32628)
map_view.camera = camera

# Función para obtener extent desde una palmera
def get_extent(row):
    camera.X = row.geometry.x
    camera.Y = row.geometry.y
    camera.scale = scale
    map_view.camera = camera
    # Necesario para dejar que la vista activa se actualice
    time.sleep(0.1)
    return map_view.camera.getExtent()

# Primer bucle por clase de ambiente (Extents únicos)
for ambiente in env_classes.keys():
    print(f"\nProcesando ambiente: {ambiente}")
    clase_df = data_env[data_env["Tip_amb"] == ambiente].copy()

    # Se mezclan las filas aleatoriamente del dataframe, permitiendo seleccionar distintas palmeras por iteración
    clase_df = clase_df.sample(frac=1, random_state=42).reset_index()
    used_extents = []       # Extent de las palmeras ya procesadas
    processed_ids = set()   # Identificadores de palmeras ya procesadas
    saved = 0               # Contador de capturas guardadas

    # Crear carpeta de capturas para este ambiente y sus worldfiles
    ambiente_folder = os.path.join(capture_folder, ambiente)
    os.makedirs(ambiente_folder, exist_ok=True)
    worldfile_folder = os.path.join(ambiente_folder, "worldfiles")
    os.makedirs(worldfile_folder, exist_ok=True)

    # Bucle hasta alcanzar el máximo por clase o hasta que no se puedan capturar más sin solaparse
    while saved < max_per_class:
        new_captures = 0    # Contador de nuevas capturas en esta ronda

        for idx, row in clase_df.iterrows():
            # Si ya se ha procesado esta palmera, se salta
            if row["index"] in processed_ids:
                continue
            
            extent = get_extent(row)
            capture_box = shapely.geometry.box(extent.XMin, extent.YMin, extent.XMax, extent.YMax)
            
            # Se comprueba si se solapa con algún extent ya capturado
            if any(capture_box.intersects(prev) for prev in used_extents):
                continue

            # Se exporta la imagen de la vista activa actual
            image_name = f"{ambiente}_{row['index']}.png"
            image_path = os.path.join(ambiente_folder, image_name)
            map_view.exportToPNG(image_path, img_size, img_size, world_file=True)

            # Se mueve el worldfile a su carpeta
            pgw_src = image_path.replace(".png", ".pgw")
            pgw_dst = os.path.join(worldfile_folder, os.path.basename(pgw_src))
            if os.path.exists(pgw_src):
                shutil.move(pgw_src, pgw_dst)
            else:
                print(f"Worldfile no encontrado para: {image_name}")

            # Se guarda la captura para saber si se ha procesado esta palmera
            used_extents.append(capture_box)
            processed_ids.add(row["index"])
            saved += 1
            new_captures += 1
            print(f"{ambiente} - Captura {saved}/{max_per_class}")

            # Si se alcanzó el máximo, detener el bucle
            if saved >= max_per_class:
                break

        # Si no se lograron nuevas capturas únicas, salir del bucle
        if new_captures == 0:
            print(f"No fue posible alcanzar {max_per_class} capturas únicas para la clase '{ambiente}'. Solo se capturaron {saved}.")
            break

    # Segunda pasada permitiendo solapamiento para completar clases que no lleguen al máximo
    if saved < max_per_class:

        print(f"Intentando completar la clase '{ambiente}' permitiendo solapamientos")
        
        for idx, row in clase_df.iterrows():
            if row["index"] in processed_ids:
                continue
            
            # Se obtiene el extent centrado en la palmera (aunque ahora no se verifica solapamiento)
            extent = get_extent(row)
            image_name = f"{ambiente}_{row['index']}.png"
            image_path = os.path.join(ambiente_folder, image_name)
            map_view.exportToPNG(image_path, img_size, img_size, world_file=True)

            pgw_src = image_path.replace(".png", ".pgw")
            pgw_dst = os.path.join(worldfile_folder, os.path.basename(pgw_src))
            if os.path.exists(pgw_src):
                shutil.move(pgw_src, pgw_dst)

            processed_ids.add(row["index"])
            saved += 1
            print(f"{ambiente} - Captura solapada {saved}/{max_per_class}")

            if saved >= max_per_class:
                print(f"'{ambiente}' completado con imágenes solapadas.")
                break
    

    if saved >= max_per_class:
        print(f"Clase '{ambiente}' completada con {saved} capturas únicas.")


print("Se finaliza el proceso de captura de imágenes.")

Crea la división de los datos de entrenamiento, validación y prueba.

In [ ]:
output_folder = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\environment_classification_dataset"
os.makedirs(output_folder, exist_ok=True)

capture_folder = os.path.join(output_folder, "CapturesRaw")
os.makedirs(capture_folder, exist_ok=True)

folders = {
    "train": os.path.join(output_folder, "Train"),
    "val": os.path.join(output_folder, "Val"),
    "test": os.path.join(output_folder, "Test")
}

islands = ["GRAN CANARIA", "TENERIFE", "LA GOMERA"]

data_islands = data[data["ISLA"].isin(islands)].copy()
data_env = data_islands[data_islands["Tip_amb"].notna()].copy()

# Rellenar los nulos con "pc"
data_islands["Hibrid"] = data_islands["Hibrid"].fillna("pc")
data_type = data_islands.copy()

# Crear diccionarios de clases
env_classes = {k: i for i, k in enumerate(sorted(data_env["Tip_amb"].unique())) if k != "?"}
type_classes = {k: i for i, k in enumerate(sorted(data_type["Hibrid"].unique())) if k != "?"}

# Bucle por cada clase de ambiente para distribuir las imágenes en Train/Val/Test
for ambiente in env_classes.keys():
    ambiente_src = os.path.join(capture_folder, ambiente)

    # Se listan las imágenes png en la carpeta correspondiente a este ambiente
    images = [f for f in os.listdir(ambiente_src) if f.endswith(".png")]
    
    # Mezcla aleatoria de las imágenes (seed para reproducibilidad)
    random.seed(42)
    random.shuffle(images)

    # Cálculo de la cantidad de imágenes para cada conjunto (70/20/10)
    n_total = len(images)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.2)
    n_test = n_total - n_train - n_val

    # Guardar en diccionario cuántas imágenes irán a cada conjunto
    split_counts = {"Train": n_train, "Val": n_val, "Test": n_test}
    split_images = {
        "Train": images[:n_train],
        "Val": images[n_train:n_train + n_val],
        "Test": images[n_train + n_val:]
    }

    # Copia de las imágenes a sus carpetas correspondientes
    for split_name in ["Train", "Val", "Test"]:
        out_dir = os.path.join(output_folder, split_name, ambiente)
        os.makedirs(out_dir, exist_ok=True)

        for img in split_images[split_name]:
            src = os.path.join(ambiente_src, img)
            dst = os.path.join(out_dir, img)
            shutil.copy2(src, dst)

        print(f"{split_name}/{ambiente}: {split_counts[split_name]} imágenes")

print("\nDataset preparado copiando en estructura Train / Val / Test.")

Mismo patrón, pero para el dataset de tipos de palmera.

In [ ]:
output_folder = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\palm_type_classification_dataset"
os.makedirs(output_folder, exist_ok=True)

capture_folder = os.path.join(output_folder, "captures_raw")
os.makedirs(capture_folder, exist_ok=True)

folders = {
    "train": os.path.join(output_folder, "Train"),
    "val": os.path.join(output_folder, "Val"),
    "test": os.path.join(output_folder, "Test")
}

islands = ["GRAN CANARIA", "TENERIFE", "LA GOMERA"]

data_islands = data[data["ISLA"].isin(islands)].copy()

# Rellenar los nulos con "pc" (nulos son las palmeras que no son híbridas, es decir, son palmeras canarias)
data_islands["Hibrid"] = data_islands["Hibrid"].fillna("pc")
data_type = data_islands.copy()

# Crear diccionarios de clases
type_classes = {k: i for i, k in enumerate(sorted(data_type["Hibrid"].unique())) if k != "?"}

print("type_classes: ", type_classes)


# Parámetros para las RoI
resolution_m_per_px = 0.25
scale = 1000
img_size = 45
max_per_class = 850

# Reproyección de las coordenadas de las palmeras a UTM Zona 28N
data_type = data_type.to_crs(epsg=32628)

# Conexión con ArcGIS Pro y su vista activa
project = arcpy.mp.ArcGISProject("CURRENT")
map_view = project.activeView
camera = map_view.camera
camera.spatialReference = arcpy.SpatialReference(32628)
map_view.camera = camera

# Función para obtener extent desde una palmera
def get_extent(row):
    camera.X = row.geometry.x
    camera.Y = row.geometry.y
    camera.scale = scale
    map_view.camera = camera
    # Necesario para dejar que la vista activa se actualice
    time.sleep(0.1)
    return map_view.camera.getExtent()

# Bucle por clase de tipo de palmera
for palm_type in type_classes.keys():

    print(f"\nProcesando tipo de palmera: {palm_type}")
    # Filtrar el dataframe por el tipo de palmera actual
    clase_df = data_islands[data_islands["Hibrid"] == palm_type].copy()
    if clase_df.empty:
        print(f"No hay palmeras para el tipo: '{palm_type}'")
        continue

    # Mezclar aleatoriamente las palmeras para variar las capturas
    clase_df = clase_df.sample(frac=1, random_state=42).reset_index()
    processed_ids = set()       # Identificadores de palmeras ya procesadas
    saved = 0                   # Contador de capturas guardadas    

    # Se crea carpeta para esta clase de palmera y su worldfile
    palm_type_folder = os.path.join(capture_folder, palm_type)
    os.makedirs(palm_type_folder, exist_ok=True)
    worldfile_folder = os.path.join(palm_type_folder, "worldfiles")
    os.makedirs(worldfile_folder, exist_ok=True)

    # Bucle por palmeras de esta clase hasta alcanzar el máximo permitido
    for idx, row in clase_df.iterrows():
        if row["index"] in processed_ids:
            continue
            
        # Obtención del extent centrado en la palmera y captura del RoI
        extent = get_extent(row)
        image_name = f"{palm_type}_{row['index']}.png"
        image_path = os.path.join(palm_type_folder, image_name)
        map_view.exportToPNG(image_path, img_size, img_size, world_file=True)

        # Se mueve el worldfile a su carpeta
        pgw_src = image_path.replace(".png", ".pgw")
        pgw_dst = os.path.join(worldfile_folder, os.path.basename(pgw_src))
        if os.path.exists(pgw_src):
            shutil.move(pgw_src, pgw_dst)
        else:
            print(f"Worldfile no encontrado para: {image_name}")

        # Se guarda la palmera como procesada
        processed_ids.add(row["index"])
        saved += 1
        print(f"{palm_type} - Captura {saved}/{max_per_class}")

        if saved >= max_per_class:
            break

    print(f"Tipo '{palm_type}' completado con {saved} capturas.")

De la misma manera, se mezclan las imágenes capturadas y se forman los subconjuntos Train/Val/Test

In [ ]:
output_folder = r"F:\Universidad\Curso 2024-25\Segundo Semestre\TFG\Desarrollo\dataset\classification_per_type\palm_type_classification_dataset"
os.makedirs(output_folder, exist_ok=True)  # Crear la carpeta principal si no existe

# Carpeta donde se encuentran las capturas generadas (por tipo de palmera)
capture_folder = os.path.join(output_folder, "captures_raw")
os.makedirs(capture_folder, exist_ok=True)

# Diccionario con las rutas donde se almacenarán las divisiones Train/Val/Test
folders = {
    "train": os.path.join(output_folder, "Train"),
    "val": os.path.join(output_folder, "Val"),
    "test": os.path.join(output_folder, "Test")
}

# Islas seleccionadas para generar el dataset
islands = ["GRAN CANARIA", "TENERIFE", "LA GOMERA"]
data_islands = data[data["ISLA"].isin(islands)].copy()  # Filtrar solo esas islas

# Rellenar los valores nulos en el campo 'Hibrid' (tipo de palmera) con 'pc'
data_islands["Hibrid"] = data_islands["Hibrid"].fillna("pc")
data_type = data_islands.copy()

# Crear diccionario de clases para el tipo de palmera (ignorando clases con valor "?")
type_classes = {k: i for i, k in enumerate(sorted(data_type["Hibrid"].unique())) if k != "?"}

# Iterar por cada clase de tipo de palmera
for palm_type in type_classes.keys():
    palm_type_src = os.path.join(capture_folder, palm_type)
    if not os.path.exists(palm_type_src):
        continue 

    # Obtener todas las imágenes png correspondientes a esta clase
    images = [f for f in os.listdir(palm_type_src) if f.endswith(".png")]

    # Mezclar aleatoriamente las imágenes con seed para reproducibilidad
    random.seed(42)
    random.shuffle(images)

    # Calcular cuántas imágenes irán a cada subconjunto
    n_total = len(images)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.2)
    n_test = n_total - n_train - n_val 

    # Crear estructura de listas para cada subconjunto
    split_counts = {"Train": n_train, "Val": n_val, "Test": n_test}
    split_images = {
        "Train": images[:n_train],
        "Val": images[n_train:n_train + n_val],
        "Test": images[n_train + n_val:]
    }

    # Copiar las imágenes a las carpetas correspondientes
    for split_name in ["Train", "Val", "Test"]:
        out_dir = os.path.join(output_folder, split_name, palm_type)
        os.makedirs(out_dir, exist_ok=True)

        for img in split_images[split_name]:
            src = os.path.join(palm_type_src, img)
            dst = os.path.join(out_dir, img)
            shutil.copy2(src, dst)

        print(f"{split_name}/{palm_type}: {split_counts[split_name]} imágenes")

print("\nDataset preparado copiando en estructura Train / Val / Test.")
